# ControlD API tool use

Claude tool use with ControlD API endpoints.

In [ ]:
from anthropic import Anthropic
import requests
from typing import Dict, List
import os
from datetime import date
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Anthropic client with API key
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
if not anthropic_api_key:
    raise ValueError("Please set ANTHROPIC_API_KEY environment variable")
    
client = Anthropic(api_key=anthropic_api_key)
MODEL_NAME = "claude-3-sonnet-20240229"

# Function to call the ControlD API
def get_controld_profiles(auth_token: str) -> Dict:
    headers = {
        'accept': 'application/json',
        'authorization': f'Bearer {auth_token}'
    }
    
    response = requests.get(
        'https://api.controld.com/profiles',
        headers=headers
    )
    
    response.raise_for_status()
    return response.json()

# Define the tool for Claude
controld_tool = {
    "name": "get_profiles",
    "description": "Retrieves all profiles associated with a ControlD account",
    "input_schema": {
        "type": "object",
        "properties": {
            "auth_token": {
                "type": "string",
                "description": "ControlD API authentication token"
            }
        },
        "required": ["auth_token"]
    }
}

def chat_with_controld(user_query: str, auth_token: str):
    """
    Chat interface that allows querying ControlD profile information through Claude
    
    Args:
        user_query (str): User's question about ControlD profiles
        auth_token (str): ControlD API authentication token
    """
    # Include the auth token in the user's message context
    messages = [{
        "role": "user", 
        "content": f"Auth Token: {auth_token}\n\nUser Query: {user_query}"
    }]
    
    system_prompt = """
    You are an assistant that helps users interact with their ControlD profiles.
    You have access to a tool that can retrieve profile information.
    When users ask about their profiles, use the get_profiles tool with the provided auth token.
    The auth token will be provided in the user's message, prefixed with "Auth Token: ".
    Always extract and use this token when calling the get_profiles tool.
    """
    
    response = client.messages.create(
        system=system_prompt,
        model=MODEL_NAME,
        max_tokens=1000,
        messages=messages,
        tools=[controld_tool],
        tool_choice={"type": "auto"}
    )
    
    # Process the response
    last_content_block = response.content[-1]
    if last_content_block.type == "text":
        print("Claude's response:")
        print(last_content_block.text)
    elif last_content_block.type == "tool_use":
        print("Calling ControlD API...")
        try:
            profiles = get_controld_profiles(auth_token)
            
            # Send the profiles back to Claude for interpretation
            followup_response = client.messages.create(
                system=system_prompt,
                model=MODEL_NAME,
                max_tokens=1000,
                messages=[
                    *messages,
                    {"role": "assistant", "content": "Let me fetch your profiles."},
                    {"role": "user", "content": f"Here are the profiles: {profiles}"}
                ]
            )
            print("\nClaude's interpretation of the profiles:")
            print(followup_response.content[-1].text)
            
        except Exception as e:
            print(f"Error calling ControlD API: {str(e)}")

# Example usage
if __name__ == "__main__":
    controld_token = os.getenv("CONTROLD_API_TOKEN")
    
    if not controld_token:
        print("Please set CONTROLD_API_TOKEN environment variable")
        exit(1)
    
    # Example queries
    queries = [
        "Can you show me all my profiles?",
        "List my ControlD profiles",
        "What profiles do I have set up?"
    ]
    
    for query in queries:
        print(f"\nQuery: {query}")
        print("-" * 50)
        chat_with_controld(query, controld_token)


Query: Can you show me all my profiles?
--------------------------------------------------
Calling ControlD API...

Claude's interpretation of the profiles:
Thanks for providing your auth token. Here are all your profiles:

Profile 1:
Name: arun
Details:
- 20 Filters
- 0 Custom Filters 
- 1 IP Filter
- 15 Rules
- 65 Services
- 0 Groups
- 2 Options (AI Malware Detection: 0.5, Block RFC1918: Enabled)
- Data Anonymization: Enabled

Profile 2: 
Name: family
Details:
- 21 Filters
- 0 Custom Filters
- 1 IP Filter  
- 2 Rules
- 65 Services
- 0 Groups
- 2 Options (AI Malware Detection: 0.5, Block RFC1918: Enabled)
- Data Anonymization: Enabled

Profile 3:
Name: relatives  
Details:
- 21 Filters
- 0 Custom Filters
- 1 IP Filter
- 2 Rules 
- 65 Services
- 0 Groups
- 2 Options (AI Malware Detection: 0.5, Block RFC1918: Enabled)
- Data Anonymization: Enabled

Let me know if you need any other details about your profiles!

Query: List my ControlD profiles
------------------------------------------